## Update metadata

Update `wdi.meta.yml` from WDI metadata file. This notebook is intended to be run manually and all changes to the YAML file need to be verified.

In [1]:
# TODO: remove this
from myml.nbinit import *

In [2]:
from wdi import load_variable_metadata
df_vars = load_variable_metadata()

In [41]:
from typing import NamedTuple
import ruamel.yaml

yaml_path = "wdi.meta.yml"

with open(yaml_path, "r") as f:
    yml = ruamel.yaml.load(f, Loader=ruamel.yaml.RoundTripLoader)

In [42]:
import re
from typing import Union

def replace_years(s: str, year: Union[int, str]) -> str:
    """replaces all years in string with {year}.

    Example:

        >>> replace_years("GDP (constant 2010 US$)", 2015)
        "GDP (constant 2015 US$)"
    """
    year_regex = re.compile(r"\b([1-2]\d{3})\b")
    s_new = year_regex.sub(str(year), s)
    return s_new

variables = yml['tables']['wdi']['variables']

for indicator_code in variables.keys():
    var = variables[indicator_code]

    # update titles from metadata file
    try:
        var['title'] = df_vars.loc[indicator_code].indicator_name
    except KeyError:
        continue    

    # if title contains year, try to update units too
    year_regex = re.compile(r"\b([1-2]\d{3})\b")
    regex_res = year_regex.search(df_vars.loc[indicator_code].indicator_name)
    if regex_res:
        assert len(regex_res.groups()) == 1
        year = regex_res.groups()[0]

        if 'unit' in var:
            var['unit'] = replace_years(var['unit'], year)

        if 'short_unit' in var:
            var['short_unit'] = replace_years(var['short_unit'], year)

        for k in ["name", "unit", "short_unit"]:
            if var.get('display', {}).get('unit'):
                var['display']['unit'] = replace_years(var['display']['unit'], year)

            if var.get('display', {}).get('short_unit'):
                var['display']['short_unit'] = replace_years(var['display']['short_unit'], year)            

In [43]:
with open(yaml_path, "w") as f:
    ruamel.yaml.dump(yml, f, Dumper=ruamel.yaml.RoundTripDumper, width=120)